Install libraries

In [3]:
!pip install langchain langchain-huggingface langchain-community chromadb transformers torch


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


2. Import Libraries

In [4]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import FastEmbedEmbeddings
from langchain.vectorstores import Chroma
from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import HuggingFacePipeline
from langchain.memory import ConversationBufferMemory
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import os, getpass

3. Authenticate Hugging Face

In [5]:
# Prompt for your Hugging Face API key if not already set
if not os.getenv("HUGGINGFACEHUB_API_TOKEN"):
    os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass.getpass("Enter your Hugging Face API key: ")


4. Load and Split CTSE Lecture Notes

In [6]:
# Load your lecture notes PDF file
loader = PyPDFLoader("../CTSE_Lecture_Notes.pdf")  # Replace with your file name
pages = loader.load_and_split()

# Split into chunks (important for context-aware retrieval)
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1024,
    chunk_overlap=100,
    length_function=len,
    add_start_index=True,
)
docs = splitter.split_documents(pages)
print(f"Split {len(pages)} pages into {len(docs)} chunks.")

Split 376 pages into 382 chunks.


5. Create Embeddings & Vector Store

In [7]:
persist_directory = "./chroma_langchain_db"

# Use FastEmbed to convert text into vectors
embeddings = FastEmbedEmbeddings()

if os.path.exists(persist_directory):
    # If already exists, load the existing DB
    vector_store = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
    print("Loaded existing vector store.")

else:
    # Otherwise, create and save
    vector_store = Chroma.from_documents(
        documents=docs,
        embedding=embeddings,
        persist_directory=persist_directory
    )
    vector_store.persist()
    print("Created and saved new vector store.")


C:\Users\bhanu\AppData\Local\Temp\ipykernel_15808\3614847898.py:8: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(persist_directory=persist_directory, embedding_function=embeddings)


Loaded existing vector store.


6. Set Up Retriever

In [8]:
# Retriever with threshold filtering
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={
        "k": 3,
        "score_threshold": 0.5,
    }
)

7. Initialize LLaMA 3 via Ollama

In [9]:
# Initialize LLaMA 3 via Ollama
llm = ChatOllama(model="tinyllama")

C:\Users\bhanu\AppData\Local\Temp\ipykernel_15808\3118759340.py:2: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="tinyllama")


8. Build Retrieval QA Chain

In [10]:
# Prompt using chat template
chat_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant. Use the provided context to answer the user's question. If you don't know the answer based on the context, say 'I don't know.'"),
    ("human", "Context:\n{context}\n\nQuestion: {question}"),
])


In [11]:
# Memory for multi-turn conversation
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

C:\Users\bhanu\AppData\Local\Temp\ipykernel_15808\4007295912.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [12]:
# Build Conversational Retrieval QA Chain
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": chat_template}
)

9. Define Chatbot Function

In [13]:
def ask(query: str):
    # Ask a question using the qa_chain
    result = qa_chain.invoke({"question": query})

    # Print the answer
    print("\nAnswer:", result["answer"])

    # Print document sources if available
    if "source_documents" in result:
        for doc in result["source_documents"]:
            print("Source:", doc.metadata.get("source", "Unknown"))


10. Batch Querying – Test Multiple Questions

In [14]:
ask("What are the key principles of CTSE?")

No relevant docs were retrieved using the relevance score threshold 0.5



Answer: Answer: The key principles of CTSE (computer science and technology) are:
1. Software engineering: Developing, testing, debugging, maintaining, and adapting software systems to meet specific requirements and user needs.
2. Computer architecture: Designing, implementing, and optimizing computer hardware for maximum efficiency and performance.
3. Databases: Building and managing databases that store and retrieve data using various database management tools.
4. Security: Protecting computer systems and data from unauthorized access or attack.
5. Human-computer interaction (HCI): Designing interfaces between humans and computers to facilitate efficient communication, collaboration, and decision-making.
6. Artificial intelligence (AI): Developing intelligent algorithms that can learn and adapt to new situations based on data input.
7. Cloud computing: Utilizing cloud services for storing, processing, and sharing large amounts of data over the internet.
8. Internet of Things (IoT): 